In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss, confusion_matrix, classification_report, roc_curve, auc
import gensim
from gensim.models import word2vec, KeyedVectors
from gensim.test.utils import common_texts
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import pickle
drive.mount('/drive')

Mounted at /drive


In [8]:
data_type = 'Long' # All, Short, Med,Long
if data_type == 'All':
  df_reviews = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/IMDB_Review_w_Clean.json')
  df_kfold = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/StratifiedKfold_index.json')
else:
  df_reviews = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/'+data_type.lower()+'_clean_data.json')
  df_kfold = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/StratifiedKfold('+data_type.lower()+')_index.json')


In [ ]:
# load word2vec model
w2vmodel = KeyedVectors.load_word2vec_format("/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Word Embedding/w2v_model/w2v_sg1_window5_100d.txt")
model = w2vmodel


In [ ]:
def text_vector(texts):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    if len(texts) == 0:
      print("empty text")
    filtered_texts = [word for word in texts if word in model.wv.vocab]
    if len(filtered_texts) == 0:
      print("empty filtered_text")
      print(texts)
      print(filtered_texts)
      return np.zeros((100,), dtype=float)
    else:
      return np.mean(w2vmodel[filtered_texts], axis=0)

In [9]:
for i in range(0,5):
  index_of_kfold = i # 0-4
  X_train, X_test = df_reviews[['clean_review_text']].iloc[df_kfold["train_index"][index_of_kfold]], df_reviews[['clean_review_text']].iloc[df_kfold["test_index"][index_of_kfold]]
  y_train, y_test = df_reviews["is_spoiler"].iloc[df_kfold["train_index"][index_of_kfold]], df_reviews["is_spoiler"].iloc[df_kfold["test_index"][index_of_kfold]]

  lb = LabelEncoder()
  y_train_encoded = lb.fit_transform(y_train)
  y_test_encoded = lb.fit_transform(y_test)

  X_train['text_vector'] = X_train.clean_review_text.apply(text_vector)
  X_test['text_vector'] = X_test.clean_review_text.apply(text_vector)

  # lb = LabelEncoder()
  # y_train_encoded = lb.fit_transform(y_train)

  X_train_vector = list(X_train['text_vector'])
  X_test_vector = list(X_test['text_vector'])
  clfmodel = LogisticRegression(C=100)
  clfmodel.fit(X_train_vector, y_train_encoded)
  # clfmodel.fit(X_train_vector, y_train_encoded)

  y_train_pred = clfmodel.predict(X_train_vector)
  y_test_pred = clfmodel.predict(X_test_vector)
  print("====================Train=============================")
  mnb_score = accuracy_score(y_train_encoded, y_train_pred)
  auc_score = roc_auc_score(y_train_encoded,y_train_pred)
  print(mnb_score, auc_score)
  print(classification_report(y_train_encoded, y_train_pred))
  print(confusion_matrix(y_train_encoded, y_train_pred))
  print("====================Test=============================")
  mnb_score = accuracy_score(y_test_encoded, y_test_pred)
  auc_score = roc_auc_score(y_test_encoded,y_test_pred)
  print(mnb_score, auc_score)
  print(classification_report(y_test_encoded, y_test_pred))
  print(confusion_matrix(y_test_encoded, y_test_pred))


  df_evaluation  = pd.DataFrame(index=X_test.index)
  df_evaluation['LogReg(no_sample,w2v)'] = y_test_pred
  df_evaluation.to_csv("/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Test_data/"+data_type+"_Data/Sample"+str(index_of_kfold)+"/logreg(no_sample,w2v).csv")

  
  filename = '/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Finished_model/'+data_type+'_Data/Sample'+str(index_of_kfold)+'/logreg(no_sample,w2v)_sample'+str(index_of_kfold)+'.sav'
  pickle.dump(clfmodel, open(filename, 'wb'))

  # load the model from disk
  loaded_model = pickle.load(open(filename, 'rb'))
  result = loaded_model.score(X_test_vector, y_test_encoded)
  print(result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


====================Train=============================
0.6718850769483681 0.6485279568905273
              precision    recall  f1-score   support

           0       0.68      0.80      0.74     45497
           1       0.65      0.49      0.56     33582

    accuracy                           0.67     79079
   macro avg       0.67      0.65      0.65     79079
weighted avg       0.67      0.67      0.66     79079

[[36559  8938]
 [17009 16573]]
====================Test=============================
0.6580677794638341 0.6300307602077672
              precision    recall  f1-score   support

           0       0.67      0.82      0.73     11374
           1       0.64      0.44      0.52      8396

    accuracy                           0.66     19770
   macro avg       0.65      0.63      0.63     19770
weighted avg       0.65      0.66      0.64     19770

[[9283 2091]
 [4669 3727]]
0.6580677794638341


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


====================Train=============================
0.6671050468518823 0.642533417739948
              precision    recall  f1-score   support

           0       0.68      0.81      0.74     45497
           1       0.65      0.48      0.55     33582

    accuracy                           0.67     79079
   macro avg       0.66      0.64      0.64     79079
weighted avg       0.66      0.67      0.66     79079

[[36653  8844]
 [17481 16101]]
====================Test=============================
0.6800202326757714 0.6513234613215215
              precision    recall  f1-score   support

           0       0.68      0.84      0.75     11374
           1       0.68      0.46      0.55      8396

    accuracy                           0.68     19770
   macro avg       0.68      0.65      0.65     19770
weighted avg       0.68      0.68      0.67     19770

[[9575 1799]
 [4527 3869]]
0.6800202326757714


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


====================Train=============================
0.6658531342075645 0.6414181395657986
              precision    recall  f1-score   support

           0       0.68      0.80      0.73     45497
           1       0.64      0.48      0.55     33582

    accuracy                           0.67     79079
   macro avg       0.66      0.64      0.64     79079
weighted avg       0.66      0.67      0.66     79079

[[36561  8936]
 [17488 16094]]
====================Test=============================
0.6790591805766313 0.6548384633576256
              precision    recall  f1-score   support

           0       0.69      0.82      0.75     11374
           1       0.66      0.49      0.57      8396

    accuracy                           0.68     19770
   macro avg       0.68      0.65      0.66     19770
weighted avg       0.68      0.68      0.67     19770

[[9277 2097]
 [4248 4148]]
0.6790591805766313


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


====================Train=============================
0.670430834987797 0.6462636763120053
              precision    recall  f1-score   support

           0       0.68      0.81      0.74     45496
           1       0.65      0.49      0.56     33583

    accuracy                           0.67     79079
   macro avg       0.66      0.65      0.65     79079
weighted avg       0.67      0.67      0.66     79079

[[36701  8795]
 [17267 16316]]
====================Test=============================
0.6643904906423874 0.6436919149938805
              precision    recall  f1-score   support

           0       0.68      0.78      0.73     11375
           1       0.63      0.51      0.56      8395

    accuracy                           0.66     19770
   macro avg       0.66      0.64      0.64     19770
weighted avg       0.66      0.66      0.66     19770

[[8884 2491]
 [4144 4251]]
0.6643904906423874


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


====================Train=============================
0.6742539200809307 0.6510272382993563
              precision    recall  f1-score   support

           0       0.68      0.81      0.74     45497
           1       0.65      0.50      0.56     33583

    accuracy                           0.67     79080
   macro avg       0.67      0.65      0.65     79080
weighted avg       0.67      0.67      0.67     79080

[[36634  8863]
 [16897 16686]]
====================Test=============================
0.6507157671101219 0.6308771517707596
              precision    recall  f1-score   support

           0       0.67      0.76      0.72     11374
           1       0.61      0.50      0.55      8395

    accuracy                           0.65     19769
   macro avg       0.64      0.63      0.63     19769
weighted avg       0.65      0.65      0.64     19769

[[8673 2701]
 [4204 4191]]
0.6507157671101219
